In [1]:
import sys
sys.path.append('../')

In [2]:
from simulate import simulate_surgeries
from util import *
import seaborn as sns
import lightgbm as lgbm
import joblib

In [3]:
# # simulation
df = simulate_surgeries(70000)
continuous_columns = ['age', 'surgery_complexity']
ordinal_columns = ['gender', 'race', 'marital_status', 'specialties']
date_columns = ['scheduled_date']
target_column = 'is_cancelled'

# dummy dataset
# df = sns.load_dataset('titanic')
# continuous_columns = ['age', 'fare', 'sibsp', 'parch']
# ordinal_columns = ['sex', 'class']
# date_columns = []
# target_column = 'survived'

# optimization params
hyperparams_stepwise_groups = [
    ['learning_rate'],
    ['lambda_l1'], 
    ['lambda_l2'], 
    ['num_leaves'], 
    ['feature_fraction'], 
    ['bagging_fraction'], 
    ['bagging_freq'],
    ['min_child_samples'],
]
fixed_params={
        'verbosity': -1,
        'objective': 'binary',
        'is_unbalance': True,
        'n_estimators': 1000
    }
save_dir = '../runs/test_run'

In [4]:
best_params, best_value = get_lgbm_optimized_params(
    df,
    continuous_columns,
    ordinal_columns,
    date_columns,
    target_column,
    save_dir,
    hyperparams_stepwise_groups=hyperparams_stepwise_groups,
    # hyperparams_stepwise_groups=None,
    fixed_params=fixed_params,
    n_trials=10,
    random_state=0
    )

[I 2024-04-14 11:23:00,927] A new study created in memory with name: no-name-6997a2d0-b93e-4a1a-b931-6cd6202102b8


optimizing group: ['learning_rate']


[I 2024-04-14 11:23:16,409] Trial 0 finished with value: 0.6366734778034112 and parameters: {'learning_rate': 0.1}. Best is trial 0 with value: 0.6366734778034112.
[I 2024-04-14 11:23:32,312] Trial 1 finished with value: 0.6540141742563363 and parameters: {'learning_rate': 0.044303752452182654}. Best is trial 1 with value: 0.6540141742563363.
[I 2024-04-14 11:23:46,803] Trial 2 finished with value: 0.6288813214177031 and parameters: {'learning_rate': 0.13981961408994045}. Best is trial 1 with value: 0.6540141742563363.
[W 2024-04-14 11:23:54,026] Trial 3 failed with parameters: {'learning_rate': 0.06431172050131989} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\PendragonS\Miniconda3\envs\general-ml\Lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "c:\Users\PendragonS\repos\general-ml\notebooks\..\util.py", line 182, in objective
    

KeyboardInterrupt: 

In [ ]:
X, y = preprocess(df, continuous_columns, ordinal_columns, date_columns, target_column)
model = lgbm.LGBMClassifier(**best_params)
joblib.dump(model, '../runs/test_run/best_model.pkl')

['../runs/test_run/best_model.pkl']